### Libraries

In [26]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qclib.gates.ldmcu import Ldmcu
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
# from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [2]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [39]:
def built_circuit(pauli_string='x', error=0.1, approximated = True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base, 'controls')
    target = QuantumRegister(1, 'target')
    circ = QuantumCircuit(controls, target)
    circ.x(list(range(len(controls)+1)))
    
    if approximated:
        MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
        Ldmcu.ldmcu(circ, pauli_matrix_dagger, controls, target)
        
    circ.measure_all()
    
    return circ

In [40]:
approx_circuit = built_circuit('x', 0.1)
print(approx_circuit)

            ┌───┐┌──────────────┐ ░ ┌─┐                  
controls_0: ┤ X ├┤0             ├─░─┤M├──────────────────
            ├───┤│              │ ░ └╥┘┌─┐               
controls_1: ┤ X ├┤1             ├─░──╫─┤M├───────────────
            ├───┤│              │ ░  ║ └╥┘┌─┐            
controls_2: ┤ X ├┤2             ├─░──╫──╫─┤M├────────────
            ├───┤│              │ ░  ║  ║ └╥┘┌─┐         
controls_3: ┤ X ├┤3 Ldmcuapprox ├─░──╫──╫──╫─┤M├─────────
            ├───┤│              │ ░  ║  ║  ║ └╥┘┌─┐      
controls_4: ┤ X ├┤4             ├─░──╫──╫──╫──╫─┤M├──────
            ├───┤│              │ ░  ║  ║  ║  ║ └╥┘┌─┐   
controls_5: ┤ X ├┤5             ├─░──╫──╫──╫──╫──╫─┤M├───
            ├───┤│              │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
    target: ┤ X ├┤6             ├─░──╫──╫──╫──╫──╫──╫─┤M├
            └───┘└──────────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
    meas: 7/═════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                     0  1  2  3  4  5  6 


In [41]:
real_circuit = built_circuit('x', 0.1, False)
print(real_circuit)

            ┌───┐┌────────┐ ░ ┌─┐                  
controls_0: ┤ X ├┤0       ├─░─┤M├──────────────────
            ├───┤│        │ ░ └╥┘┌─┐               
controls_1: ┤ X ├┤1       ├─░──╫─┤M├───────────────
            ├───┤│        │ ░  ║ └╥┘┌─┐            
controls_2: ┤ X ├┤2       ├─░──╫──╫─┤M├────────────
            ├───┤│        │ ░  ║  ║ └╥┘┌─┐         
controls_3: ┤ X ├┤3 Ldmcu ├─░──╫──╫──╫─┤M├─────────
            ├───┤│        │ ░  ║  ║  ║ └╥┘┌─┐      
controls_4: ┤ X ├┤4       ├─░──╫──╫──╫──╫─┤M├──────
            ├───┤│        │ ░  ║  ║  ║  ║ └╥┘┌─┐   
controls_5: ┤ X ├┤5       ├─░──╫──╫──╫──╫──╫─┤M├───
            ├───┤│        │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
    target: ┤ X ├┤6       ├─░──╫──╫──╫──╫──╫──╫─┤M├
            └───┘└────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
    meas: 7/═══════════════════╩══╩══╩══╩══╩══╩══╩═
                               0  1  2  3  4  5  6 


In [72]:
from qiskit.quantum_info import DensityMatrix
from qiskit_aer.noise import NoiseModel

In [75]:
def density_matrix(circuit):
    size = len(circuit)
    # Generate a size-qubit simulated backend
    backend = GenericBackendV2(num_qubits=size)
    
    # Transpile the circuit to a circuit that can be directly executed by the backend
    transpiled_circuit = transpile(circuit, backend)
    # transpiled_circuit.save_density_matrix() -> método antigo da API
    
    noise_model = NoiseModel.from_backend(backend)
    
    
    # Run the transpiled circuit using the simulated fake backend
    job = backend.run(transpiled_circuit)
    result = job.result()
    
    # Get the density matrix
    d_m = result.data(0)['density_matrix']
    return d_m

In [69]:
rc_density_matrix = density_matrix(real_circuit)

D:\UFPE\env_paper\qclib-papers\Lib\site-packages\qiskit\providers\fake_provider\generic_backend_v2.py:506: RuntimeWarning: Aer not found using BasicSimulator and no noise
  warnings.warn("Aer not found using BasicSimulator and no noise", RuntimeWarning)


KeyError: 'density_matrix'

In [ ]:
ac_density_matrix = density_matrix(approx_circuit)

### Adicionar medida de fidelidade.

In [74]:
from qiskit.quantum_info import state_fidelity

In [96]:
c1 = QuantumCircuit(3)
c1.x(range(3))
c1.ccx(0,1,2)
c1.measure_all()
c1.draw('text')

┌───┐      ░ ┌─┐      
   q_0: ┤ X ├──■───░─┤M├──────
        ├───┤  │   ░ └╥┘┌─┐   
   q_1: ┤ X ├──■───░──╫─┤M├───
        ├───┤┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ┤ X ├┤ X ├─░──╫──╫─┤M├
        └───┘└───┘ ░  ║  ║ └╥┘
meas: 3/══════════════╩══╩══╩═
                      0  1  2